함수형 프로그래밍은 지연 또는 엄격하지 않은 연산 순서를 강조한다. 답을 계산할 때 컴파일러나 런타임이 가능하면 최소한의 작업만을 수행하게 만들자는 것이 아이디어다. 파이썬은 엄격한 평가 순서를 강제하는 경향이 있다.

if, elif, else문을 생각해보자. 이들의 의미는 분명하며, 일기도 쉽다. 하지만 이 문장들은 조건을 평가할 때 특정 순서대로 실행될 것임을 암시한다. 이 경우에도 어느 정도는 엄격한 연순 순서에서 벗어날 수 있고, 제약이 있긴 하지만 엄격하지 않은 조건문을 만들 수 이싿. 이러한 기능이 유용할 것인지 분명히 알 수는 없지만, 그러한 조건문은 알고리즘을 함수적인 스타일로 표현할 때 취할 수 있는 몇 가지 대안을 보여준다.

첫 번째 부분은 엄격하지 않은 평가를 구현하는 방법을 살펴보는 것이다. 그러한 방법은 성능 최적화를 가져올 수 있기 때문에 관심을 가질 만한 도구다. 앞에서 몇 가지 고차 함수를 살펴봤다. 어떤 경우에는 고차 함수를 사용해 상당히 복잡한 함수를 데이터 컬렉션에 적용했다. 다른 경우에는 간단한 함수를 데이터 컬렉션에 적용했다.

실제로는 다인 파이썬 연산자를 함수에 적용하기 위해 매우 작은 람다 객체를 작성하는 경우가 자주 생긴다. 예를 들어 다음과 같이 proc() 함수를 정했다.

In [3]:
import functools

prod = lambda iterable: functools.reduce(lambda x, y: x*y, iterable, 1)

In [4]:
prod((1, 2, 3))

6

이 구문을 더 단순하게 만들 수 있을까? operator 모듈은 내장 연산자에 대한 정의를 제공한다. 

operator 모듈이 제공하는 몇 가지 기능은 고차 함수의 생성을 간단하고 더 명확하게 해준다. 개념적으로는 중요하지만 operator 모듈은 첫눈에는 그리 흥미로워 보이지는 않는다.

### 조건식 평가하기

파이썬은 식을 계산할 때 상대적으로 엄격한 순서를 지킨다. 이 규칙의 가장 유명한 예외는 and와 or이다. 파이썬은 명령의 평가에도 매우 엄격한 순서를 적용한다. 그로 인해 이러한 엄격한 평가 순서를 피하기 위한 다른 방법을 찾는 것은 도전적인 일이다. 

조건 연산을 평가하는 것이 명령어의 엄격하지 않은 수행 순서를 가지고 실험할 수 있는 한 가지 방법이라고 할 수 있다. if와 else문을 리팩터링하여 파이썬의 엄격하지 않은 평가와 관련된 부분을 알아본다.

if, elif else문은 첫 번째 문장부터 끝 문장까지 순서대로 실행된다. 이상적으로는, 최적화 컴파일러가 조건식을 평가하는 가장 빠른 순서를 찾을 수 있도록 언어에서 이 규칙을 완화할 수도 있다. 여기서 아이디어는 실제 평가 순서는 엄격하지 않을지라도 코드를 보는 독자들이 더 이해하기 쉬운 순서로 조건문을 작성할 수 있어야 한다는 것이다.

최적화 컴파일러가 없기 때문에 파이썬은 이러한 개념과는 조금 거리가 멀다. 그렇지만 명령형으로 문장을 실행하는 대신, 함수를 평가하는 것으로 조건을 기술하는 방법이 존재한다. 이렇게 하면 실행 시점에 원하는 대로 평가 순서를 바꿀 수 있다.

파이썬에는 조건식인 if와 else가 있다. 이 식은 조건이 하나만 있는 경우 사용할 수 있다. 하지만 조건이 둘 이상 있다면 문장이 이상스럽게 복잡해진다. 하위 식을 내포시킬 때 조심해야 하기 때문이다. 그러한 경우 식을 이해하기가 너무 복잡하기 때문에 결국 명령문으로 해당 식을 대치하게 된다.

In [7]:
n = 2
x = 1
y = 2
z = 3
(x if n==1 else (y if n==2 else z))

2

딕셔너리 키와 람다를 사용해 매우 복잡한 조건을 만들 수 있다. 다음은 계승 함수를 식으로 표현한 것이다.

In [27]:
def fact(n):
    f = {n == 0: lambda n:1, n ==1: lambda n:1, n==2: lambda n:2, n>2: lambda n:fact(n-1)*n}[True]
    return f(n)

첫 번째 단계에서는 여러 가지 조건을 평가한다. 주어지 조건 중 하나가 True가 되고, 나머지는 False가 될 것이다. 결과로 생기는 딕셔너리에는 두 가지 원소가 들어 간다. 그중 True인 람다를 성택하여 f 변수에 대입한다. 

매핑에 람다를 사용한 이유는 딕셔너리를 만드는 시점에 값을 계산하기를 원치 한기 때문이다. 

In [28]:
fact(4)

24

### 엄격하지 않은 딕셔너리 규칙 활용하기

딕셔너리 키에는 순서가 없다. 키 갑싱 같은 원소가 여러개 존재하는 딕셔러니를 만든다면, 결과 dict 객체에는 그중 하나의 원소만 들어간다. 키가 중복되는 값중 어떤 것이 남을지는 분명치 않지만, 그 사실이 문제가 되지는 않아야 한다.

다음은 어떤 원소가 남아도 관계 업다는 것이 명확한 경우다.

In [29]:
def max(a, b):
    f = {a >= b: lambda: a, b >= a: lambda: b}[True]
    return f()

### 참인 조건식 걸러내기

어떤 식이 True인지 결정하는 방법은 여러 가지가 있다. 앞의 예에서는 키에 따른 람다 값을 딕셔너리에 넣었다. 딕셔너리를 구축하는 과정에서 키가 True인 것 중 한 가지 값만 살아남는다.

다음은 같은 상황을 filter() 함수를 사용해 다르게 기술한 것이다.

In [30]:
def semifact(n):
    alternative = [(n==0, lambda n:1), (n==1, lambda n:1), (n==2, lambda n:2), (n>2, lambda n: semifact(n-2)*n)]
    c, f = next(filter(itemgetter(0), alternative))
    return f(n)

여러 대안을 조건과 함수의 쌍으로 표현했다. filter 함수에 itemgetter(0) 매개변수를 적용하면, 조건이 True인 쌍을 골라낼 수 있다. 그 후 조건이 True인 반복 가능 객체에서 첫 번째 원소를 선택했다. 이때 조건은 c에, 함수는 f에 대입한다. 조건은 무시해도 되고(True임) filter를 적용해 얻은 f함수를 평가해야 한다.

딕셔너리를 사용한 예제와 마찬가지로 여기서도 람다를 사용해 조건을 평가할 때 식이 평가되지 않도록 지연시켰다.

semifact 함수를 이중 계승이라고도 부른다.

### 람다 대신 operator 모듈 사용하기

max, min, sorted 함수를 사용하는 경우, key= 매개변수를 지정할 수도 있다. 이 함수는 각 고차 함수의 동작을 변경시키는 인자 값이다. 보통은 튜플에서 원소를 가져오는 간단한 람다를 사용했다. 다음은 우리가 자주 사용했던 두 가지 예제다.

In [1]:
fst = lambda x: x[0]
snd = lambda x: x[1]

사실 이러한 함수는 operator 모듈에 정의가 들어 있다.

다은은 예제에 사용할 데이터다.

In [2]:
year_cheese = [(2000, 29.87), (2001, 30.12), (2002, 30.6), (2003, 30.66), (2004, 31.33), (2005, 32.62), (2006, 32.73), 
              (2007, 33.5), (2008, 32.84), (2009, 33.02), (2010, 32.92)]

연간 치즈 사용량이다. 다음 명령을 사용해 치즈 사용량이 가장 작은 지점을 찾을 수 있다.

In [3]:
min(year_cheese, key= lambda x: x[1])

(2000, 29.87)

In [4]:
min(year_cheese, key= snd)

(2000, 29.87)

fst, snd 와 같은 함수를 직접 정의하는 대신, 다음과 같이 itemgetter(0)과 itemgetter(1)을 사용할 수 있다.

In [5]:
from operator import *

max(year_cheese, key=itemgetter(1))

(2007, 33.5)

itemgetter 함수는 \__getitem__() 특별 메서드를 호출해 인덱스를 지정해 튜플에서 원소를 가져온다.

### 고차 함수를 사용할 때 이름 있는 애트리뷰트 가져오기

이름이 없는 튜플 대신, 이름 있는 튜플을 사용한다고 가정해보자. 다음과 같이 치즈 소비량을 이름 있는 튜플로 만들 수 있다.

In [6]:
from collections import namedtuple

YearCheese = namedtuple("YearCheese", ("year", "cheese"))
year_cheese_2 = list(YearCheese(*yc) for yc in year_cheese)

In [7]:
year_cheese_2

[YearCheese(year=2000, cheese=29.87),
 YearCheese(year=2001, cheese=30.12),
 YearCheese(year=2002, cheese=30.6),
 YearCheese(year=2003, cheese=30.66),
 YearCheese(year=2004, cheese=31.33),
 YearCheese(year=2005, cheese=32.62),
 YearCheese(year=2006, cheese=32.73),
 YearCheese(year=2007, cheese=33.5),
 YearCheese(year=2008, cheese=32.84),
 YearCheese(year=2009, cheese=33.02),
 YearCheese(year=2010, cheese=32.92)]

람다나 attrgetter 함수를 사용하면 사용량을 가져올 수 있다.

In [8]:
min(year_cheese_2, key=attrgetter('cheese'))

YearCheese(year=2000, cheese=29.87)

In [9]:
max(year_cheese_2, key=lambda x: x.cheese)

YearCheese(year=2007, cheese=33.5)

attrgetter는 문자열을 함수의 인자로 받을 수도 있을 것이다. 따라서 훨씬 더 유연한 코드를 작성할 수 있다.

### 연산자를 사용한 starmap

itertools.starmap 함수는 값의 쌍으로 이뤄진 시퀀스에 대해 연산자를 적용한다. 다음 예제를 살펴보자.

In [13]:
from itertools import starmap 
from itertools import zip_longest 

d = starmap(pow, zip_longest([], range(4), fillvalue=60))

In [15]:
list(d)

[1, 60, 3600, 216000]

In [16]:
zip_longest([], range(4), fillvalue=60)

In [17]:
list(_)

[(60, 0), (60, 1), (60, 2), (60, 3)]

fillvalue 매개변수는 더 짧은 시퀀스의 데이터를 모두 사용한 후에 빈 자리를 채워넣을 때 쓰인다.

### 연산자를 사용해 축약하기

연산자 정의를 사용하는 방법을 하나 더 살펴보자. 연산자를 functools.reduce 함수에 사용할 수 있다. 예를 들어 sum 함수를 다음과 같이 정의할 수 있다.

In [19]:
import functools
import operator

sum = functools.partial(functools.reduce, operator.add)


곱을 계산하는 비슷한 함수가 필요하다면, 다음과 같이 정의할 수 있다.

In [21]:
prod = functools.partial(functools.reduce, operator.mul)

우아하지만 대부분의 경우 operator 함수를 직접 사용할 일은 없다. 그러나 람다를 적용하여 우아하게 적용할 수 있다.